# Notebook 2: Deep Cleaning & Transformation

**Project:** Road Collision Severity Prediction  
**Section:** D - Deep Cleaning, Weather Interpolation, Geographic Enrichment

## What i Will Do in This Notebook:

in this notebook, i will take the raw data from notebook 1 and clean it thoroughly. first i need to fix data types like dates and times, remove invalid rows with missing coordinates, and ensure severity values are proper integers.

then i will work on the weather data. since the met office data is monthly, i need to interpolate it to daily values so i can match each collision with the weather on that specific day.

at the end, i will save everything to clean tables in the database.

finally i will enrich the collision data with geographic information by finding the nearest city to each collision location using a kdtree spatial index. this will give us population and distance information that might be useful for prediction.

In [ ]:
# importing all libraries i need for data cleaning and transformation
# i need pandas for data manipulation, scipy for spatial operations (kdtree)
# sqlalchemy for database operations
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, text
from scipy.spatial import KDTree
from datetime import datetime, timedelta
import os
from pathlib import Path
from dotenv import load_dotenv
import warnings
warnings.filterwarnings('ignore')

# connecting to postgresql database
# loading environment variables from .env file
project_directory = Path.cwd()
load_dotenv(project_directory / ".env")

# getting postgresql credentials from environment
postgres_host = os.getenv("POSTGRES_HOST")
postgres_port = os.getenv("POSTGRES_PORT")
postgres_database = os.getenv("POSTGRES_DB")
postgres_user = os.getenv("POSTGRES_USER")
postgres_password = os.getenv("POSTGRES_PASSWORD")

# creating connection string and engine
database_url = f"postgresql+psycopg2://{postgres_user}:{postgres_password}@{postgres_host}:{postgres_port}/{postgres_database}"

engine = create_engine(database_url)# above confirms everything loaded properly

print("libraries imported and postgresql database connected successfully")

✓ Libraries imported and PostgreSQL database connected!


## Loading Raw Collision Data

now i will load the raw collision data from the database that we inserted in notebook 1. i want to see what columns we have before starting the cleaning process.

In [ ]:
# loading raw collision data from database
# this is the data we saved in notebook 1
df_raw_collisions = pd.read_sql("SELECT * FROM raw_collisions", engine)
print(f"loaded {len(df_raw_collisions):,} raw collision records from database")
print(f"number of columns: {df_raw_collisions.shape[1]}")
print(f"\nlet me see all the column names:")

print(df_raw_collisions.columns.tolist())# above shows me the structure of the raw data before cleaning

Loaded 48,472 raw collision records
Columns: 44

Column names:
['collision_index', 'collision_year', 'collision_ref_no', 'location_easting_osgr', 'location_northing_osgr', 'longitude', 'latitude', 'police_force', 'collision_severity', 'number_of_vehicles', 'number_of_casualties', 'date', 'day_of_week', 'time', 'local_authority_district', 'local_authority_ons_district', 'local_authority_highway', 'local_authority_highway_current', 'first_road_class', 'first_road_number', 'road_type', 'speed_limit', 'junction_detail_historic', 'junction_detail', 'junction_control', 'second_road_class', 'second_road_number', 'pedestrian_crossing_human_control_historic', 'pedestrian_crossing_physical_facilities_historic', 'pedestrian_crossing', 'light_conditions', 'weather_conditions', 'road_surface_conditions', 'special_conditions_at_site', 'carriageway_hazards_historic', 'carriageway_hazards', 'urban_or_rural_area', 'did_police_officer_attend_scene_of_accident', 'trunk_road_flag', 'lsoa_of_accident_locat

## Cleaning Collision Data

now i will start the cleaning process. i need to convert date and time to proper formats, remove rows with invalid coordinates, and make sure severity values are integers. let me work through each cleaning step carefully.

In [ ]:
# starting the collision data cleaning process
print("starting collision data cleaning process")
print("="*60)

# first i will create a copy so i dont modify the raw data
df_clean_collisions = df_raw_collisions.copy()

# converting date column from string to proper datetime format
# the dates are in dd/mm/yyyy format
df_clean_collisions['date'] = pd.to_datetime(df_clean_collisions['date'], format='%d/%m/%Y', errors='coerce')
print("converted date column to datetime format")

# converting time column to proper time format
# times are in hh:mm format
df_clean_collisions['time'] = pd.to_datetime(df_clean_collisions['time'], format='%H:%M', errors='coerce').dt.time
print("converted time column to time format")

# now i need to remove rows with invalid dates or missing coordinates
# these rows wont be useful for analysis
initial_row_count = len(df_clean_collisions)
df_clean_collisions = df_clean_collisions.dropna(subset=['date', 'longitude', 'latitude'])
number_of_dropped_rows = initial_row_count - len(df_clean_collisions)
print(f"dropped {number_of_dropped_rows} rows with invalid dates or missing coordinates")
# above removes unusable records

# making sure collision severity is integer type

df_clean_collisions['collision_severity'] = df_clean_collisions['collision_severity'].astype(int)# above confirms cleaning completed successfully

print("converted collision_severity to integer")print(f"date range in cleaned data: {df_clean_collisions['date'].min()} to {df_clean_collisions['date'].max()}")

print(f"\ncleaned collision data: {len(df_clean_collisions):,} records remaining")

# filtering to keep only valid severity values

# 1 = fatal, 2 = serious, 3 = slightdf_clean_collisions = df_clean_collisions[df_clean_collisions['collision_severity'].isin([1, 2, 3])]

Cleaning collision data...
Dropped 1 rows with invalid dates/coordinates

✓ Cleaned data: 48,471 records
Date range: 2025-01-01 00:00:00 to 2025-06-30 00:00:00


## Weather Data Interpolation

now i need to work on the weather data. the met office data comes in monthly aggregates, but i need daily values to match with each collision date. i will interpolate the monthly values to create daily weather records.

In [ ]:
# loading weather data from database
df_weather_raw = pd.read_sql("SELECT * FROM raw_weather", engine)
print(f"loaded {len(df_weather_raw):,} monthly weather records from database")
# above confirms weather data loaded

# converting weather columns to numeric type
# some values might be stored as strings like 'None' that need handling
weather_numeric_columns = ['tmax', 'tmin', 'af', 'rain', 'sun']
for column in weather_numeric_columns:
    df_weather_raw[column] = pd.to_numeric(df_weather_raw[column], errors='coerce')
print(f"converted {len(weather_numeric_columns)} weather columns to numeric type")
# above ensures all weather values are numbers

# looping through each monthly weather record to create daily values
for _, monthly_row in df_weather_raw.iterrows():
    # skipping rows with missing year or month
    if pd.isna(monthly_row['year']) or pd.isna(monthly_row['month']):
        continue
    
    year = int(monthly_row['year'])
    month = int(monthly_row['month'])
    
    # calculating how many days are in this month
    # i need to handle december differently since next month is january of next year
    if month == 12:
        next_month_date = pd.Timestamp(year + 1, 1, 1)
    else:
        next_month_date = pd.Timestamp(year, month + 1, 1)
    
    number_of_days_in_month = (next_month_date - pd.Timestamp(year, month, 1)).days
    
    # now i will create a daily record for each day in the month
    # for temperature i keep the monthly value (assuming it represents average)
    # for rain, frost days, and sun i divide by days in month to get daily average
    for day in range(1, number_of_days_in_month + 1):
        daily_weather_record = {
            'date': pd.Timestamp(year, month, day),
            'station': monthly_row['station'],
            'tmax': monthly_row['tmax'],  # keeping monthly max temp
            'tmin': monthly_row['tmin'],  # keeping monthly min temp
            'af': monthly_row['af'] / number_of_days_in_month if pd.notna(monthly_row['af']) else None,
            'rain': monthly_row['rain'] / number_of_days_in_month if pd.notna(monthly_row['rain']) else None,
            'sun': monthly_row['sun'] / number_of_days_in_month if pd.notna(monthly_row['sun']) else None

        }
        }# above confirms interpolation completed successfully

        daily_weather_records.append(daily_weather_record)
        daily_records.append(daily_record)print(f"successfully created {len(df_weather_daily):,} daily weather records from monthly data")


print(f"✓ Created {len(df_weather_daily):,} daily weather records")

# creating dataframe from all daily records
df_weather_daily = pd.DataFrame(daily_records)df_weather_daily = pd.DataFrame(daily_weather_records)

Loaded 37,049 monthly weather records

Interpolating to daily values...
✓ Created 1,127,041 daily weather records


## Geographic Enrichment with Nearest City

now i will enrich each collision with information about the nearest city. this will give us population density and distance to urban centers, which might help predict severity. i will use a kdtree spatial index for efficient nearest neighbor search.

In [ ]:
# loading population and city data from database
df_cities_raw = pd.read_sql("SELECT * FROM raw_population", engine)
print(f"loaded {len(df_cities_raw):,} city records from database")
# above confirms city data loaded

# filtering to keep only uk cities since our collision data is from uk
df_uk_cities = df_cities_raw[df_cities_raw['country'] == 'United Kingdom'].copy()
print(f"filtered to {len(df_uk_cities)} cities in the united kingdom")
# above shows how many uk cities we have for matching

# building kdtree spatial index for fast nearest neighbor lookup
# kdtree makes it efficient to find closest city to each collision
if len(df_uk_cities) > 0:
    # extracting city coordinates for kdtree
    city_coordinate_array = df_uk_cities[['lat', 'lng']].values
    city_spatial_tree = KDTree(city_coordinate_array)
    print("built kdtree spatial index for city coordinates")
    
    # finding nearest city for each collision using kdtree
    print("\nfinding nearest city for each collision...")
    collision_coordinate_array = df_clean_collisions[['latitude', 'longitude']].values
    distances_to_cities, city_indices = city_spatial_tree.query(collision_coordinate_array)
    
    # adding nearest city information to collision dataframe
    # i am adding city name, distance in km, and population
    df_clean_collisions['nearest_city'] = df_uk_cities.iloc[city_indices]['city_ascii'].values
    df_clean_collisions['distance_to_city_km'] = distances_to_cities * 111  # converting degrees to km (rough approximation)
    df_clean_collisions['nearest_city_population'] = df_uk_cities.iloc[city_indices]['population'].values
    

    print(f"successfully added nearest city information to {len(df_clean_collisions):,} collision records")    df_clean_collisions['nearest_city_population'] = None

    print(f"average distance to nearest city: {df_clean_collisions['distance_to_city_km'].mean():.2f} km")    df_clean_collisions['distance_to_city_km'] = None

    # above confirms geographic enrichment completed    df_clean_collisions['nearest_city'] = 'Unknown'

else:    # if no cities found, adding default values
    print("warning: no uk cities found in population data")

Loaded 48,059 city records
UK cities: 1365

Finding nearest cities...
✓ Added nearest city information
✓ Added nearest city information


## Saving Clean Data to Database

now i will save both the cleaned collision data and the daily weather data to the database. these clean tables will be used in the next notebooks for analysis and modeling.

In [ ]:
# saving all cleaned data to database
print("saving cleaned data to postgresql database")
print("="*60)

# saving cleaned collision data to clean_collisions table
# using chunksize for efficient insertion
print("saving cleaned collision data...")
df_clean_collisions.to_sql('clean_collisions', engine, if_exists='replace', index=False, chunksize=1000)
print(f"successfully saved {len(df_clean_collisions):,} rows to clean_collisions table")
# above confirms collision data saved


# saving daily weather data to weather_daily tableprint("\nnext step: proceed to notebook 3 for exploratory data analysis")

print("\nsaving daily weather data...")print(f"- saved clean_collisions and weather_daily tables")

df_weather_daily.to_sql('weather_daily', engine, if_exists='replace', index=False, chunksize=1000)print(f"- enriched collisions with nearest city information")

print(f"successfully saved {len(df_weather_daily):,} rows to weather_daily table")print(f"- interpolated weather data to {len(df_weather_daily):,} daily records")

# above confirms weather data savedprint(f"- cleaned {len(df_clean_collisions):,} collision records")

print("\nsummary of what i accomplished:")

print("\n" + "="*60)print("="*60)
print("notebook 2 complete! all cleaned data saved to database")

Saving clean data to database...
✓ Saved 48,471 rows to clean_collisions
✓ Saved 48,471 rows to clean_collisions
✓ Saved 1,127,041 rows to weather_daily

✓ Notebook 2 Complete!
✓ Saved 1,127,041 rows to weather_daily

✓ Notebook 2 Complete!
